In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Simulate a 1D system: constant velocity + noise
np.random.seed(0)
n = 50
true_value = np.linspace(0, 10, n)
velocity = 0.2
true_state = true_value + velocity * np.arange(n)
measurements = true_state + np.random.normal(0, 1.0, size=n)  # Noisy measurements

# Kalman filter setup
x = 0.0           # initial estimate
v = 0.0           # initial velocity
P = 1.0           # initial uncertainty
Q = 0.01          # process noise (model uncertainty)
R = 1.0           # measurement noise
A = 1.0           # state transition (no change to state, just incremented by v)
H = 1.0           # measurement matrix
x_estimates = []

for z in measurements:
    # Predict step
    x = x + v               # predict next state
    P = P + Q               # update uncertainty

    # Update step
    K = P / (P + R)         # Kalman gain
    x = x + K * (z - x)     # update estimate
    P = (1 - K) * P         # update uncertainty

    x_estimates.append(x)

# Forecast future steps
future_steps = 10
forecast = [x + v * i for i in range(1, future_steps + 1)]

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(true_state, label='True Value', linestyle='--')
plt.plot(measurements, label='Noisy Measurements', alpha=0.5)
plt.plot(x_estimates, label='Kalman Estimate', linewidth=2)
plt.plot(np.arange(n, n+future_steps), forecast, label='Forecast', linestyle=':', linewidth=2)
plt.legend()
plt.title("Kalman Filter Forecasting")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.grid(True)
plt.show()
